# 3804ICT Data Mining
## Assignment 1: Preliminary Data Investigation Report
#### Author: Kai Sato --- Wui Theam Wong -- Ken Quach

## I. Data Exploration

In [ ]:
import pandas as pd 
import numpy as np

df = pd.read_csv("combined_listings2.csv")

### Five number summary

In [ ]:
# Helper function to clean the price
import re
def clean_price(raw):
    clean_text = raw
    clean_text = re.sub("[^0123456789\.]","",clean_text)
    return clean_text

# df['price'] = pd.to_numeric(df['price'].apply(clean_price))
# df['price'].head(5)

In [ ]:
# Five number summary
import matplotlib.pyplot as plt
col = 'reviews_per_month'
example = df[col].head()
mean = np.mean(example)
print("Min: ", df[col].min())
print("Q1 : ", np.percentile(example, 25, interpolation = 'lower'))
print("Q2 : ", np.percentile(example, 50, interpolation = 'lower'))
print("Q3 : ", np.percentile(example, 75, interpolation = 'lower'))
print("Max: ", max(df[col]))
plt.boxplot(example)
plt.title(r'boxplot')
plt.show()

### Number of listings from Sydney and Melbourne


In [ ]:
df['location'].value_counts()

### Types of rental properties


In [ ]:
df['property_type'].value_counts()

### Number of hosts that have verification on Airbnb


In [ ]:
df['host_identity_verified'].value_counts()

### The first review of all listings


In [ ]:
# convert from str to datetime
date = pd.to_datetime(df['first_review'])
print("The first review of all listings: ", date.min())

In [ ]:
df['host_since'].value_counts()

## II. Visualisation

In [ ]:

import matplotlib.pyplot as plt
import numpy as np

temp = df[['bedrooms']].copy()

temp = temp.dropna(subset=['bedrooms'])

temp['bedrooms'] = temp['bedrooms'].astype('int32')

print(temp.value_counts().head(19).sort_index())

temp.plot.hist(bins=100, bottom=0.001,title='Bedroom Frequency')

plt.show()

In [ ]:
temp = df[['price']].copy()


temp = temp.dropna(subset=['price'])
temp['price'] = temp['price'].replace('[\$,]', '', regex=True).astype(float)

print(temp.value_counts().sort_index(ascending=False))

temp.plot.hist(bins=100, bottom=0.01,title='price Frequency')

plt.show()

temp.plot.hist(bins=100, bottom=0.01,title='price Frequency with range',range=(0,1000))

plt.show()

In [ ]:
#Location
temp2 = df[['latitude','longitude','location']].copy()
temp2['latitude'] = temp2['latitude'].astype(float)
temp2['longitude'] = temp2['longitude'].astype(float)

print(temp2.head(10))

temp2[temp2['location'] == 'sydney'].plot.scatter(x = 'latitude', y ='longitude',title= "Sydney")
temp2[temp2['location'] =='melbourne'].plot.scatter(x = 'latitude', y ='longitude',title= "Melbourne" )
plt.show()

In [ ]:
temp = df[['amenities']].copy()

temp.value_counts()

array = []

amenities_total = {}
flag = False;
#go through all records
for listing in temp['amenities']:
    #go through all amenities in single listing
    listing = listing.split(",")
    for amenities in listing:
        #check for dups
        flag = False
        
        for amenitieslist in amenities_total:            
            if len(amenities.split('"')) > 1:
                if amenitieslist == amenities.split('"')[1]:
                    flag = True
        if len(amenities.split('"')) > 1:
            #if we found our amenities in the list than update total or add an item with count to 1
            
            if flag:                
                array.append(amenities.split('"')[1])
                amenities_total[amenities.split('"')[1]]= int(amenities_total[amenities.split('"')[1]]) + 1
            else:
                array.append(amenities.split('"')[1])
                amenities_total[amenities.split('"')[1]] = 1
            


In [ ]:
#

total = pd.Series(data=array)
total.value_counts().sort_values(ascending=False).head(15).plot(kind='pie',autopct='%1.1f%%')
plt.ylabel("")
plt.show()
print("total unique Amenities: "+str(len(amenities_total)))
print("-------")
print("Least used Amenities")
print(total.value_counts().sort_values(ascending=True).head(5))

In [ ]:
#avability
temp = df[['review_scores_rating']].copy()

temp.plot.hist(bins=100, bottom=0.01,title='Reviews')
plt.show()

## III. Data pre-processing


In [ ]:
# Read in data as a pandas dataframe
import pandas as pd
df = pd.read_csv("combined_listings2.csv")
t = df[['id','name','description','neighborhood_overview','host_location','host_total_listings_count','host_identity_verified','neighbourhood_cleansed','latitude','longitude','property_type','room_type','accommodates','bathrooms_text','bedrooms','beds','amenities','price','minimum_nights','maximum_nights','availability_30','availability_60','availability_90'
,'availability_365','number_of_reviews','number_of_reviews_ltm','number_of_reviews_l30d','first_review','last_review','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location'
,'review_scores_value','instant_bookable','calculated_host_listings_count','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms','reviews_per_month','location']].copy()
df = t
df.dtypes

In [ ]:
# Handle missing data
# Find which attributes have any missing data
df.isna().any()

In [ ]:
# Show Amount of rows the data now has
df.shape

In [ ]:
# Drop any duplicates in the data - if any exist
df = df.drop_duplicates()

# Show starting number of rows the dataset contains
df.shape

In [ ]:
# Remove missing/incomplete data based on attributes where they should be removed.
# To ensure proper data cleaning the attributes: bathrooms_text, bedrooms, beds, 
# review_scores_rating, review_scores_accuracy, review_scores_cleanliness, first_review, last_review
# review_scores_checkin, review_scores_communication, review_scores_location, review_scores_value, and reviews_per_month.
# will have the any entries with NaN for these attributes removed.
# Without the values of the attribute the data is not usable for the algorithm and can negatively affect the results.
df = df.dropna(subset=['bathrooms_text','bedrooms','beds',
                       'review_scores_rating','review_scores_accuracy','review_scores_cleanliness',
                       'review_scores_checkin','review_scores_communication','review_scores_location',
                       'review_scores_value','reviews_per_month','first_review','last_review'])


# Format price to remove the dollar sign and commas
df['price'] = df['price'].str.replace(',','')
df['price'] = df['price'].str.replace('$','',regex=False)
df['price'] = df['price'].str.replace(' ', '')

df.shape
# As shown by the output some missing/incomplete data was removed 
# to prevent these from negatively affecting the algorithm's results

In [ ]:
# Check all the variables being cleaned to see if there are no empty attributes data entries
# these are different from NaN as the cell is completely empty -> If true these will be removed
print("Bathrooms:", df['bathrooms_text'].empty)
print("Bedrooms:", df['bedrooms'].empty)
print("Beds:", df['beds'].empty)
print("Rating:", df['review_scores_rating'].empty)
print("Accuracy:", df['review_scores_accuracy'].empty)
print("Cleanliness:", df['review_scores_cleanliness'].empty)
print("Checkin:", df['review_scores_checkin'].empty)
print("Communication:", df['review_scores_communication'].empty)
print("Location:", df['review_scores_location'].empty)
print("Value:", df['review_scores_value'].empty)
print("Reviews per month:", df['reviews_per_month'].empty)
print("First Reviews:", df['first_review'].empty)
print("Last Reviews:", df['last_review'].empty)

In [ ]:
# Examine all the attributes for invalid data
# Check if Id is a valid number (greater than 0)
df = df[df['id'] >= 1]
# Total listing should be greater than or equal to 1
df = df[df['host_total_listings_count'] >= 1]
# Check the integer values related to the property - should be at least 1
df = df[df['bedrooms'] >= 1]
df = df[df['beds'] >= 1]

# Convert price from object to float
df = df.astype({'price': 'float64'})
# Check and remove negative numbered property listings
df = df[df['price'] >= 0]

# Minimum and Maximum nights must be at least 1
df = df[df['minimum_nights'] >= 1]
df = df[df['maximum_nights'] >= 1]

# Avaliability must be greater than or equal to 0 
df = df[df['availability_30'] >= 0]
df = df[df['availability_60'] >= 0]
df = df[df['availability_90'] >= 0]
df = df[df['availability_365'] >= 0]

# must also be no more than the number of days of their category 
# (avaliability_30 must not exceed 30 days)
df = df[df['availability_30'] <= 30]
df = df[df['availability_60'] <= 60]
df = df[df['availability_90'] <= 90]
df = df[df['availability_365'] <= 365]

# Number of reviews - should be greater than or equal to 0
df = df[df['number_of_reviews'] >= 0]
df = df[df['number_of_reviews_ltm'] >= 0]
df = df[df['number_of_reviews_l30d'] >= 0]

# Review scores - Check for incorrect values across all aspects
df = df[df['review_scores_rating'] >= 0]
df = df[df['review_scores_rating'] <= 100]

df = df[df['review_scores_accuracy'] >= 0]
df = df[df['review_scores_accuracy'] <= 10]

df = df[df['review_scores_cleanliness'] >= 0]
df = df[df['review_scores_cleanliness'] <= 10]

df = df[df['review_scores_checkin'] >= 0]
df = df[df['review_scores_checkin'] <= 10]

df = df[df['review_scores_communication'] >= 0]
df = df[df['review_scores_communication'] <= 10]

df = df[df['review_scores_location'] >= 0]
df = df[df['review_scores_location'] <= 10]

df = df[df['review_scores_value'] >= 0]
df = df[df['review_scores_value'] <= 10]

# Calculated host listings - check for invalid entries
df = df[df['calculated_host_listings_count'] >= 0]

df = df[df['calculated_host_listings_count_entire_homes'] >= 0]

df = df[df['calculated_host_listings_count_private_rooms'] >= 0]

df = df[df['calculated_host_listings_count_shared_rooms'] >= 0]

df = df[df['reviews_per_month'] >= 0]

# Print final amount of rows
df.shape

In [ ]:
# Noisy Data
# From the attributes selected to be used in the algorithm - look for outliers
# Create boxplot to find minimum and maximum of the varaibles where outliers are most likely to exist
# this will determine if there are any outliers in the data
boxplot = df.boxplot(column=['minimum_nights', 'maximum_nights', 'number_of_reviews', 'bedrooms', 'beds'])

In [ ]:
# From the five selected variables - use Z score to determine the outliers
# Create a list of all the variables
columns = list(df.columns)
# Remove all the attributes not being used (except for the five selected above)
columns.remove('id')
columns.remove('name')
columns.remove('description')
columns.remove('neighborhood_overview')
columns.remove('host_location')
columns.remove('host_total_listings_count')
columns.remove('host_identity_verified')
columns.remove('neighbourhood_cleansed')
columns.remove('latitude')
columns.remove('longitude')
columns.remove('property_type')
columns.remove('room_type')
columns.remove('accommodates')
columns.remove('bathrooms_text')
columns.remove('amenities')
columns.remove('price')
columns.remove('availability_30')
columns.remove('availability_60')
columns.remove('availability_90')
columns.remove('availability_365')
columns.remove('number_of_reviews_ltm')
columns.remove('number_of_reviews_l30d')
columns.remove('first_review')
columns.remove('last_review')
columns.remove('review_scores_rating')
columns.remove('review_scores_accuracy')
columns.remove('review_scores_cleanliness')
columns.remove('review_scores_checkin')
columns.remove('review_scores_communication')
columns.remove('review_scores_location')
columns.remove('review_scores_value')
columns.remove('instant_bookable')
columns.remove('calculated_host_listings_count')
columns.remove('calculated_host_listings_count_entire_homes')
columns.remove('calculated_host_listings_count_private_rooms')
columns.remove('calculated_host_listings_count_shared_rooms')
columns.remove('reviews_per_month')
columns.remove('location')

var = 0 # Counts the number of Z score columns created to prevent duplicate columns from being created (if code is run again)

df.shape


In [ ]:
# Calculate Z Score for each varaible and column creating a new column for each one
for column in columns:
    column_z_score = column + '_z_score'
    if var < 5: # If the five Z score columns haven't been made yet create them
        df[column_z_score] = (df[column] - df[column].mean())/df[column].std(ddof=0)
        var = var + 1
    
# Remove any rows of data in which the z score for any of them is three standard deviations away
# this is to only remove extreme outliers in the data (as seen with maximum number of nights)

df = df.loc[df['bedrooms_z_score'].abs() <= 3]

df = df.loc[df['beds_z_score'].abs() <= 3]

df = df.loc[df['minimum_nights_z_score'].abs() <= 3]

df = df.loc[df['maximum_nights_z_score'].abs() <= 3]

df = df.loc[df['number_of_reviews_z_score'].abs() <= 3]


df.shape

In [ ]:
# Boxplot showing the data with extreme outliers removed
boxplot = df.boxplot(column=['minimum_nights','maximum_nights', 'number_of_reviews', 'bedrooms', 'beds'])

In [ ]:
# Sanity Check -> ensure that one attribute links correctly with another (in the correct format)
# Convert first_review and last review to proper date/time format
df['first_review'] = pd.to_datetime(df['first_review'], format='%d/%m/%y')
df['last_review'] = pd.to_datetime(df['last_review'], format='%d/%m/%y')

# Perform sanity check - ensure that last_review is a date that is later than first_review
df = df.loc[df['first_review'] < df['last_review']]
df.shape

In [ ]:
id                                                int64
name                                             object
description                                      object
neighborhood_overview                            object
host_location                                    object
host_total_listings_count                       float64
host_identity_verified                           object
neighbourhood_cleansed                           object
latitude                                        float64
longitude                                       float64
property_type                                    object
room_type                                        object
accommodates                                      int64
bathrooms_text                                   object
bedrooms                                        float64 Outlier 4
beds                                            float64 Outlier 5
amenities                                        object
price                                            object
minimum_nights                                    int64 Outlier 1
maximum_nights                                    int64 Outlier 2
availability_30                                   int64
availability_60                                   int64
availability_90                                   int64
availability_365                                  int64
number_of_reviews                                 int64 Outlier 3
number_of_reviews_ltm                             int64 
number_of_reviews_l30d                            int64
first_review                                     object Sanity Check 1 \/
last_review                                      object Sanity Check 1 /\
review_scores_rating                            float64
review_scores_accuracy                          float64
review_scores_cleanliness                       float64
review_scores_checkin                           float64
review_scores_communication                     float64
review_scores_location                          float64
review_scores_value                             float64
instant_bookable                                 object
calculated_host_listings_count                    int64
calculated_host_listings_count_entire_homes       int64
calculated_host_listings_count_private_rooms      int64
calculated_host_listings_count_shared_rooms       int64
reviews_per_month                               float64
location                                         object